格子上の最短ベクトルの数え上げを行うアルゴリズム<br>
入力： $n$ 次元格子 $L$ の基底のGSO係数 $\mu_{i,j} (1 \leq j < i \leq n)$, 
GSOベクトルの二乗ノルム $B_i = \| \mathbb{b_i^\star} \|^2 (1 \leq i \leq n)$, 
数え上げ上界列 $R_1^2 \leq \cdots \leq R_n^2$<br>
出力：すべての $1 \leq k \leq n$ に対して $\| \pi_k(\mathbb{v}) \|^2 \leq R_{n+1-k}^2$ を満たす格子ベクトル<br>
　　　$\mathbb{v} = \sum_{i=1}^n v_i \mathbb{b_i} \in L$ の係数ベクトル $(v_1, \dots, v_n) \in \mathbb{Z}^n$ ( $v$ が存在する場合)

事前準備

In [1]:
import numpy as np
import math

def input_basis_form():
  print("Enter the dimension of the lattice.")
  n = int(input())
  print("---")
  print("Enter each basis vector of the lattice")
  B = np.array([list(map(int,input().split())) for _ in range(n)])
  return B

# 基底行列を入力するとそれらのGSOベクトルからなる行列とGSO係数からなる行列を出力するアルゴリズム
# サブルーチン用
def GSO(B):
  # Bの行数（基底ベクトルの本数）をnとする
  n = len(B)
  # Bと同じサイズのゼロ行列(浮動小数点数成分)を生成
  # ここにGSOの結果を入れていく予定
  B_star = np.zeros_like(B, dtype=float)
  # GSO係数行列のベースを作成
  mu = np.eye(n, dtype=float)

  for i in range(n):
    B_star[i] = B[i]
    for j in range(i):
      mu[i,j] = np.dot(B[i], B_star[j]) / np.dot(B_star[j], B_star[j])
      B_star[i] -= mu[i,j] * B_star[j]
  # B_star(GSOベクトル行列)とmu(GSO係数行列)を出力
  return B_star, mu

# 基底行列を入力するとそれらのGSOベクトルからなる行列とGSO係数からなる行列を出力するアルゴリズム
# メイン処理用
def GSO_main():
  B = input_basis_form()
  return GSO(B)

# GSO係数とGSOベクトルの二乗ノルムを出力するアルゴリズム
def GSO_coeffs_sqnorms(B):
  B_star,mu = GSO(B)
  B_sqnorm = np.array([np.dot(B_star[i],B_star[i]) for i in range(len(B))])
  return B_sqnorm, mu

# ランダムな整数係数フルランク行列を生成するアルゴリズム（やっつけバージョン）
def create_randint_fullrank_matrix():
  print("Enter the size of matrix")
  s = list(map(int, input().split()))
  row = s[0]
  col = s[1]
  while True:
    rand = np.random.randint(-100, 100, size=(row, col))
    if np.linalg.matrix_rank(rand) == row:
      return rand
    else:
      continue

数え上げ上界列を受け取るアルゴリズムを定義

In [7]:
def input_enum_upperbound_seq(B_sqnorm):
  print("---")
  print("Enter a parameter epsilon to define the enumeration upper bounds as:")
  print("R_k^2 = (k / n) * epsilon * ||b_1*||^2")
  epsilon = float(input())
  R_5 = B_sqnorm[0] * epsilon
  R = [(k+1)/5 * R_5 for k in range(5)]
  return R

本題のアルゴリズムを定義（サブルーチン用）

In [6]:
def enum_shortest_vec(B_sqnorm, mu, R):
  n = len(B_sqnorm)
  sigma = np.zeros(shape=(n+1,n), dtype=float)
  r = [i for i in range(n+1)]
  rho = [float(0) for i in range(n+1)]
  v = [0 for i in range(n)]
  c = [0 for i in range(n)]
  w = [0 for i in range(n)]
  v[0] = 1
  last_nonzero = 1
  k=0
  while True:
    rho[k] = rho[k+1] + (v[k] - c[k])**2 * B_sqnorm[k]
    if rho[k] <= R[n-1-k]:
      if k == 0:
        return v
      k -= 1
      r[k] = max(r[k+1], r[k])
      for i in range(r[k+1],k,-1):
        sigma[i,k] = sigma[i+1,k] + mu[i,k] * v[i]
      c[k] = -sigma[k+1,k]
      v[k] = round(c[k])
      w[k] = 1
    else:
      k += 1
      if k == n:
        return "none"
      r[k] = k
      if k >= last_nonzero:
        last_nonzero = k
        v[k] += 1
      else:
        if v[k] > c[k]:
          v[k] -= w[k]
        else:
          v[k] += w[k]
        w[k] += 1

実際に用いてみる

In [7]:
B = np.array([[63, -14, -1, 84, 61],
              [74, -20, 23, -32, -52],
              [93, -46, -19, 0, -63],
              [93, 11, 13, 60, 52],
              [33, -93, 12, 57, -2]])
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)

R_5 = B_sqnorm[0] * 0.001

R = [(k+1)/5 * R_5 for k in range(5)]
print("---")
print("The upper bound sequence is")
print(R)
print("---")
#vol = math.sqrt(np.prod(B_sqnorm))
#upper_bound = math.sqrt(5) ** math.pow(vol, 1/5)
#R = [upper_bound for i in range(5)]
v = enum_shortest_vec(B_sqnorm, mu, R)
if v != "none":
  print("The coefficent vector is")
  print(v)
  print("---")
  print("The following vector was found.")
  print(np.dot(v, B))
else:
  print("No lattice vector was found.")

The squared norms of the GSO vectors are
[1.49430000e+04 1.00737428e+04 3.01527354e+03 7.03392732e+02
 3.41094435e-10]
---
The GSO coefficient matrix is
[[ 1.          0.          0.          0.          0.        ]
 [-0.06297263  1.          0.          0.          0.        ]
 [ 0.17928127  1.07305735  1.          0.          0.        ]
 [ 0.93046912  0.31890545 -0.43777128  1.          0.        ]
 [ 0.53770996  0.33393597  0.72786965 -2.94334071  1.        ]]
---
The upper bound sequence is
[np.float64(2.9886), np.float64(5.9772), np.float64(8.9658), np.float64(11.9544), np.float64(14.943)]
---
The coefficent vector is
[-19306, -10353, 3097, 16259, 5524]
---
The following vector was found.
[ 0 -1 -1  0 -1]


標準入力対応版

In [14]:
B = input_basis_form()
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
R = input_enum_upperbound_seq(B_sqnorm)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)
print("---")
print("The upper bound sequence is")
print(R)
print("---")
v = enum_shortest_vec(B_sqnorm, mu, R)
if v != "none":
  print("The coefficent vector is")
  print(v)
  print("---")
  print("The following vector was found.")
  print(np.dot(v, B))
else:
  print("No lattice vector was found.")

Enter the dimension of the lattice.
---
Enter each basis vector of the lattice
---
Enter a parameter epsilon to define the enumeration upper bounds as:
R_k^2 = (k / n) * epsilon * ||b_1*||^2
The squared norms of the GSO vectors are
[5.50000000e+01 9.09090909e-01 9.91006512e-30 8.44734328e+02
 1.60422714e+04]
---
The GSO coefficient matrix is
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.27272727e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 6.36363636e-01  6.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.74545455e+00  8.60000000e+00 -1.74766553e+15  1.00000000e+00
   0.00000000e+00]
 [ 4.46181818e+01  1.25100000e+02  1.65131986e+16  1.74689243e+01
   1.00000000e+00]]
---
The upper bound sequence is
[np.float64(6.050000000000001), np.float64(12.100000000000001), np.float64(18.150000000000002), np.float64(24.200000000000003), np.float64(30.250000000000004)]
---
The coefficent vector is
[-1, 1, 0

ランダムなフルランク行列（基底行列）を生成し、その基底で張られる格子上の短いベクトルを見つけるアルゴリズム

In [10]:
B = create_randint_fullrank_matrix()
print("The basis matrix is")
print(B)
print("---")
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
R = input_enum_upperbound_seq(B_sqnorm)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)
print("---")
print("The upper bound sequence is")
print(R)
print("---")
v = enum_shortest_vec(B_sqnorm, mu, R)
if v != "none":
  print("The coefficent vector is")
  print(v)
  print("---")
  print("The following vector was found.")
  print(np.dot(v, B))
else:
  print("No lattice vector was found.")

Enter the size of matrix
The basis matrix is
[[  70   62   88   14   75]
 [ -69   28  -10  -60   25]
 [ -62  -53  -27 -100   74]
 [ -43  -65  -43    0    6]
 [  78  -20   92  -48   11]]
---
---
Enter a parameter epsilon to define the enumeration upper bounds as:
R_k^2 = (k / n) * epsilon * ||b_1*||^2
The squared norms of the GSO vectors are
[22309.          9482.81451432 10473.6656105   1092.74647755
  3318.55848296]
---
The GSO coefficient matrix is
[[ 1.          0.          0.          0.          0.        ]
 [-0.13174055  1.          0.          0.          0.        ]
 [-0.26231566  1.06962487  1.          0.          0.        ]
 [-0.46501412  0.03799753  0.44008722  1.          0.        ]
 [ 0.55892241 -0.21769139  0.46143057 -2.00795314  1.        ]]
---
The upper bound sequence is
[np.float64(2230.9), np.float64(4461.8), np.float64(6692.7), np.float64(8923.6), np.float64(11154.5)]
---
The coefficent vector is
[0, 0, 0, 1, 0]
---
The following vector was found.
[-43 -65 -43  

見つからないと悲しいので、見つからなくなるギリギリまでRを減らすアルゴリズムにしてみる

In [16]:
B = create_randint_fullrank_matrix()
print("The basis matrix is")
print(B)
print("---")
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)
print("---")

n = len(B)
epsilon = 1
min_v= [0 for i in range(len(B))]
min_v[0] = 1
while True:
  R_n = B_sqnorm[0] * epsilon
  R = [(k+1)/5 * R_n for k in range(n)]
  v = enum_shortest_vec(B_sqnorm, mu, R)
  if v != "none":
    min_v = v
    epsilon -= 0.001
    continue
  else:
    break
print("We use the following upper bound sequence as:")
print("R_k^2 = (k / n) * epsilon * ||b_1*||^2")
print("almost minimal epsilon is")
print(epsilon)
print("---")
print("The upper bound sequence is")
print(R)
print("---")
print("The coefficent vector is")
print(min_v)
print("---")
print("The following vector was found.")
print(np.dot(min_v, B))

Enter the size of matrix
The basis matrix is
[[  3  31 -20  86 -96 -55  50  11 -98  -9]
 [ 32  59  63 -94  51 -96 -86  79  79  45]
 [  1 -37  -4  97  48 -58 -14  71  39 -33]
 [-98 -57  56  55 -97  14 -80 -52 -24  75]
 [ 37 -38 -14  53  16  97  30 -73 -21 -93]
 [ 57  70 -21  61 -44  -3  45  47  92 -50]
 [ 50  15 -58 -54 -83  89  74  81   3 -10]
 [-27  71  59  91   4   0 -38  78  -1 -52]
 [-43 -98 -23 -43  25  31  58  25 -97  65]
 [ 74  47  86  27  41 -20 -92 -52 -45  56]]
---
The squared norms of the GSO vectors are
[33313.         40630.34313931 23172.17115387 40778.25399085
  7411.18871526 18679.80312627 15197.79803381 12782.21670335
  2073.73233381  8586.4931765 ]
---
The GSO coefficient matrix is
[[ 1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-0.55873083  1.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.07252424  0.15392175  1.          0.          0.    